<a href="https://colab.research.google.com/github/ViFLara/titanic_machine_learning_model/blob/master/titanic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Reading Database

In [ ]:
data = pd.read_csv('/content/train.csv')
data = data.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

In [ ]:
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,male,22.0,1,0,7.2500
1,2,1,1,female,38.0,1,0,71.2833
2,3,1,3,female,26.0,0,0,7.9250
3,4,1,1,female,35.0,1,0,53.1000
4,5,0,3,male,35.0,0,0,8.0500


In [ ]:
data = data.set_index(['PassengerId'])
data = data.rename(columns= {'Survived': 'Target'}, inplace= False)

In [ ]:
data.head()

,target,Pclass,Sex,Age,SibSp,Parch,Fare
PassengerId,,,,,,,
1,0,3,male,22.0,1,0,7.2500
2,1,1,female,38.0,1,0,71.2833
3,1,3,female,26.0,0,0,7.9250
4,1,1,female,35.0,1,0,53.1000
5,0,3,male,35.0,0,0,8.0500


# Descriptive

In [ ]:
data.describe()

,target,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
data.describe(include=['O'])

,Sex
count,891
unique,2
top,male
freq,577


# Data Transformation

In [ ]:
data['Sex_F'] = np.where(data['Sex'] == 'famale', 1, 0)
data['Pclass_1'] = np.where(data['Pclass'] == 1, 1, 0)
data['Pclass_2'] = np.where(data['Pclass'] == 2, 1, 0)
data['Pclass_3'] = np.where(data['Pclass'] == 3, 1, 0)

In [ ]:
data = data.drop(['Pclass', 'Sex'], axis=1)

In [ ]:
data.head()

,target,Age,SibSp,Parch,Fare,Sex_F,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,
1,0,22.0,1,0,7.2500,0,0,0,1
2,1,38.0,1,0,71.2833,0,1,0,0
3,1,26.0,0,0,7.9250,0,0,0,1
4,1,35.0,1,0,53.1000,0,1,0,0
5,0,35.0,0,0,8.0500,0,0,0,1


In [ ]:
data.isnull().sum()

target        0
Age         177
SibSp         0
Parch         0
Fare          0
Sex_F         0
Pclass_1      0
Pclass_2      0
Pclass_3      0
dtype: int64

In [ ]:
data.fillna(0, inplace= True)
data.isnull().sum()

target      0
Age         0
SibSp       0
Parch       0
Fare        0
Sex_F       0
Pclass_1    0
Pclass_2    0
Pclass_3    0
dtype: int64

# Sampling

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.drop(['target'], axis=1),
                                                     data['target'],
                                                     test_size = 0.3,
                                                     random_state = 1234)

[{'train': x_train.shape}, {'test': x_test.shape}]

[{'train': (623, 8)}, {'test': (268, 8)}]

# Model

In [ ]:
rndforest = RandomForestClassifier(n_estimators= 1000,
                                   criterion= 'gini',
                                   max_depth= 5)

rndforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
probability = rndforest.predict_proba(data.drop('target', axis= 1))[:, 1]
classification = rndforest.predict(data.drop('target', axis= 1))

In [ ]:
data['probability'] = probability
data['classification'] = classification

In [ ]:
data

,target,Age,SibSp,Parch,Fare,Sex_F,Pclass_1,Pclass_2,Pclass_3,probability,classification
PassengerId,,,,,,,,,,,
1,0,22.0,1,0,7.2500,0,0,0,1,0.191075,0
2,1,38.0,1,0,71.2833,0,1,0,0,0.770049,1
3,1,26.0,0,0,7.9250,0,0,0,1,0.220549,0
4,1,35.0,1,0,53.1000,0,1,0,0,0.710154,1
5,0,35.0,0,0,8.0500,0,0,0,1,0.179235,0
...,...,...,...,...,...,...,...,...,...,...,...
887,0,27.0,0,0,13.0000,0,0,1,0,0.301855,0
888,1,19.0,0,0,30.0000,0,1,0,0,0.566041,1
889,0,0.0,1,2,23.4500,0,0,0,1,0.417433,0
